In [1]:
# Import libraries
import pandas as pd
import sys
import os
import warnings
import matplotlib.pyplot as plt

from src.data_preparation_automl import load_data
from pycaret.time_series import *

# Set options for displaying dataframes and figures
pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore", category=FutureWarning)
pd.options.mode.chained_assignment = None

# Forecasting AutoML with PyCaret

In below code of the notebook, we will use pycaret to create a forecast for the sales data.

In [2]:
# Load the data
df_data = load_data()

#### Create Experiment

In [3]:
# Create the forecasting experiment
exp = TSForecastingExperiment()
exp.setup(data = df_data,
        target = "Sales_Qty",
        coverage = 0.90,
        fold_strategy = "sliding", # We are using a sliding and not an expanding window
        fold = 5, # Choosing the number of folds
        fh = 60)

,Description,Value
0,session_id,8654
1,Target,Sales_Qty
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(825, 1)"
5,Transformed data shape,"(825, 1)"
6,Transformed train set shape,"(765, 1)"
7,Transformed test set shape,"(60, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,SlidingWindowSplitter


#### Plot Cross Validation

In [4]:
# Cross validation plot
exp.plot_model(plot = "cv")

#### Plot Train and Test Split

In [5]:
# Plot train and test split
exp.plot_model(plot = "train_test_split")

#### Compare the models based on RMSE and other obtained metrics

In [6]:
# Check which one is the best model based on RMSE
exp.compare_models(sort = "rmse")

,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2,TT (Sec)
omp_cds_dt,Orthogonal Matching Pursuit w/ Cond. Deseasonalize & Detrending,1.2082,0.9688,58.4794,76.2808,10.1021,0.7977,-0.0926,0.2040
lasso_cds_dt,Lasso w/ Cond. Deseasonalize & Detrending,1.2091,0.9699,58.5173,76.3641,10.1007,0.7983,-0.0956,0.2360
llar_cds_dt,Lasso Least Angular Regressor w/ Cond. Deseasonalize & Detrending,1.2091,0.9699,58.5173,76.3641,10.1007,0.7983,-0.0956,0.2120
en_cds_dt,Elastic Net w/ Cond. Deseasonalize & Detrending,1.2091,0.9699,58.5175,76.3642,10.1007,0.7983,-0.0956,0.2300
lr_cds_dt,Linear w/ Cond. Deseasonalize & Detrending,1.2091,0.9699,58.5176,76.3644,10.1007,0.7983,-0.0956,0.2500
ridge_cds_dt,Ridge w/ Cond. Deseasonalize & Detrending,1.2091,0.9699,58.5176,76.3644,10.1007,0.7983,-0.0956,0.2180
br_cds_dt,Bayesian Ridge w/ Cond. Deseasonalize & Detrending,1.2091,0.9699,58.5168,76.3646,10.1015,0.7982,-0.0956,0.2080
polytrend,Polynomial Trend Forecaster,1.2180,0.9731,58.9227,76.6051,10.1127,0.8029,-0.1039,0.0100
croston,Croston,1.2317,0.9840,59.6548,77.5438,10.0134,0.8143,-0.1224,0.0080
gbr_cds_dt,Gradient Boosting w/ Cond. Deseasonalize & Detrending,1.2931,0.9877,62.4243,77.7227,11.9390,0.7995,-0.1371,0.2480


BaseCdsDtForecaster(fe_target_rr=[WindowSummarizer(lag_feature={'lag': [2, 1]},
                                                   n_jobs=1)],
                    regressor=OrthogonalMatchingPursuit(), sp=2,
                    window_length=2)

In the output one can see that almost all the models are deseasonalised and detrended. This basically means that the time series has been transformed in order to become stationary. However all the R-Squared values for the models are negative. This means a simple average is outbiding all the models. The models do not deviade strongly from each other in terms of the obtained metrics. Lets continue with a simple model, the theta forecaster.

A Theta forecaster is a simple and commonly used forecasting method. It falls under the category of exponential smoothing methods, which are based on the principle of giving more weight to recent observations and less weight to older observations.

The Theta method essentially forecasts future values by considering the trend component of the time series data. It's particularly effective when the data doesn't exhibit a clear seasonality but shows a linear trend.

#### Theta Forecaster

In [7]:
# Create the theta model
theta = exp.create_model("theta")

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,2015-04-11,1.1273,0.8950,53.3202,68.9682,16.4631,1.3392,-0.0555
1,2015-06-10,1.5170,1.0285,67.5129,77.3642,6.1265,0.6824,-0.5749
2,2015-08-09,1.2192,0.9832,56.2845,75.8048,8.3084,0.7728,-0.0035
3,2015-10-08,1.4489,1.1051,73.3490,91.6838,0.7714,0.8183,-1.2993
4,2015-12-07,1.6269,1.3510,85.1131,109.3407,9.0719,0.9893,-0.1366
Mean,NaT,1.3879,1.0726,67.1159,84.6323,8.1482,0.9204,-0.4140
SD,NaT,0.1865,0.1549,11.5806,14.3968,5.0699,0.2320,0.4866


In [8]:
# Tune the model
tuned_theta = exp.tune_model(theta)

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,2015-04-11,1.1273,0.8950,53.3202,68.9682,16.4631,1.3392,-0.0555
1,2015-06-10,1.5170,1.0285,67.5129,77.3642,6.1265,0.6824,-0.5749
2,2015-08-09,1.2192,0.9832,56.2845,75.8048,8.3084,0.7728,-0.0035
3,2015-10-08,1.4489,1.1051,73.3490,91.6838,0.7714,0.8183,-1.2993
4,2015-12-07,1.6269,1.3510,85.1131,109.3407,9.0719,0.9893,-0.1366
Mean,NaT,1.3879,1.0726,67.1159,84.6323,8.1482,0.9204,-0.4140
SD,NaT,0.1865,0.1549,11.5806,14.3968,5.0699,0.2320,0.4866


Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished


Even when we tuned the Theta Forecaster, the metrics stay the same and do not improve. So tuning was not really successful.

#### Diagnostics

In [9]:
exp.plot_model(estimator=tuned_theta, plot="diagnostics")

#### In-Sample Forecasting

In [10]:
exp.plot_model(tuned_theta, plot="insample")

#### Out-of-Sample Forecasting

In [11]:
exp.plot_model(tuned_theta, plot="forecast", data_kwargs={"fh":10})

#### Predict Model

The R-Squared is negative because it seems that the model is doing worse than a simple average. If we print out the predictions one can see that each prediction is basically the same value.

In [12]:
holdout_pred = exp.predict_model(tuned_theta)

,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,Theta Forecaster,0.8235,0.6262,43.1995,49.9403,0.8325,0.5074,-0.5657


In [14]:
holdout_pred.head(10)

,y_pred
2016-02-06,106.4570
2016-02-07,108.1869
2016-02-08,106.4861
2016-02-09,108.2165
2016-02-10,106.5152
2016-02-11,108.2461
2016-02-12,106.5443
2016-02-13,108.2756
2016-02-14,106.5734
2016-02-15,108.3052


#### Finalize Model

Now we finalize the model and are using all the data available. This is the last step. The goal ist to train on the complete dataset.

In [15]:
final_model = exp.finalize_model(tuned_theta)

In [17]:
exp.plot_model(final_model, plot="forecast", data_kwargs={"fh":30})